In [ ]:
# math
import numpy as np
import pandas as pd

# plotly
import plotly.graph_objects as go

# other
import copy

from experiments_2022 import IMAGE_PATH
from experiments_2022.zone_level_analysis import (
    base,
    cleaning,
    viz,
    regression_functions,
)
from experiments_2022.datasets import (
    load_zones,
    load_weather,
    pull_from_dataset,
)

from utils import (
    PROJECTS_2022,
    PROJECTS_2022_TOTAL,
    PROJECTS_VAV,
    NO_WEEKENDS,
    SUMMER_START_2022,
    SUMMER_END_2022,
    DOMINANT_COLORS,
    SINGLE_PLOT_TXT_SIZE,
    SINGLE_PLOT_LEGEND_SIZE,
    SINGLE_PLOT_ANNOTATION_SIZE,
    MULTI_PLOT_TXT_SIZE,
    MULTI_PLOT_TITLE_SIZE,
    MULTI_PLOT_LEGEND_SIZE,
    MULTI_PLOT_ANNOTATION_SIZE,
    add_vertical_boxes,
    add_line_to_subplots,
    get_2022_control_data,
    run_group_regressions,
    add_total_to_dfs,
    add_total_to_results_dict,
)

In [ ]:
DOMINANT_ZONES = {}
for project in PROJECTS_2022:
    dzs = pd.read_csv(f"./dicts_dfs/dominant/{project}.csv").set_index("Unnamed: 0")
    dzs.index.name = None
    DOMINANT_ZONES[project] = dzs

In [ ]:
DOMINANT_ZONES_TOT = add_total_to_results_dict(DOMINANT_ZONES)

## Deviation

In [ ]:
deviation = pull_from_dataset("2022", PROJECTS_2022, "zone-deviation_coolsp")

zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=deviation[project]
    )

df_filter = cleaning.create_sp_filter(zonal_schedules, sps=[74], reverse_filter=False)

control_deviation = cleaning.clean_dfs(
    dfs=deviation,
    this_var="zone-deviation_coolsp",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=True,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
    df_filter=df_filter,
)

In [ ]:
control_deviation_mean = base.run_passive_test_on_dfs(
    dfs=control_deviation, this_test="Mean", col_name="T - CSP [°C]<br>Control Days"
)
control_deviation_std = base.run_passive_test_on_dfs(
    dfs=control_deviation, this_test="Std", col_name="T - CSP [°C]<br>Control Days"
)

In [ ]:
dzs = copy.deepcopy(DOMINANT_ZONES)
for project in PROJECTS_2022:
    control_deviation_mean[project] = control_deviation_mean[project].dropna()
    control_deviation_std[project] = control_deviation_std[project].dropna()
control_deviation_mean, control_deviation_std, dzs = base.make_common_index(
    [control_deviation_mean, control_deviation_std, dzs]
)

In [ ]:
control_deviation_mean_tot = add_total_to_results_dict(control_deviation_mean)
control_deviation_std_tot = add_total_to_results_dict(control_deviation_std)
dzs_tot = add_total_to_results_dict(dzs)

In [ ]:
fig = viz.make_dot_plot(
    y_data=control_deviation_mean_tot,
    y_error_up_data=control_deviation_std_tot,
    y_error_down_data=control_deviation_std_tot,
    y_axis_title="T - CSP [°C]<br>Control Days",
    x_axis_title="Fraction of Zones [Unitless]",
    color_data=dzs_tot,
    color_legend={
        "name": {
            0: "Dominated",
            1: "Dominant",
            2: "Rogue",
        },
        "color": {
            0: DOMINANT_COLORS["Dominated"],
            1: DOMINANT_COLORS["Dominant"],
            2: DOMINANT_COLORS["Rogue"],
        },
    },
    normalize_x=True,
    num_cols=3,
    horizontal_spacing=0.1,
    vertical_spacing=0.1,
    y_range=[-5, 5],  # [-8, 8],
    width=800,
    height=500,
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
control_deviation_tot = add_total_to_dfs(control_deviation)

In [ ]:
summary = pd.DataFrame(index=PROJECTS_2022_TOTAL, columns=["Mean", "Std"])
for project in PROJECTS_2022_TOTAL:
    summary.loc[project, "Mean"] = control_deviation_tot[project].mean(axis=1).mean()
    summary.loc[project, "Std"] = control_deviation_tot[project].mean(axis=1).std()

In [ ]:
p = 0
for i in range(1, 4):  # Columns
    for j in range(1, 5):  # Rows
        if p > 10:
            continue
        project = PROJECTS_2022_TOTAL[p]
        fig.add_annotation(
            text=f"Building Wide T - CSP: {round(summary.loc[project, 'Mean'], 1)} +/- {round(summary.loc[project, 'Std'], 1)} °C",
            xref=f"x{p+1}",
            yref=f"y{p+1}",
            x=-0.05,
            y=4,
            xanchor="left",
            yanchor="top",
            showarrow=False,
            font=dict(size=MULTI_PLOT_ANNOTATION_SIZE, color="black"),
        )
        p += 1

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC5.png")

## Tloads

In [ ]:
tloads = pull_from_dataset("2022", PROJECTS_2022, "zone-tloads")

zonal_schedules = {}
for project in PROJECTS_2022:
    zonal_schedules[project] = regression_functions.get_zonal_sp_schedule(
        project, experiment_year="2022", freq="hourly", df=tloads[project]
    )

df_filter = cleaning.create_sp_filter(zonal_schedules, sps=[74], reverse_filter=False)

control_tloads = cleaning.clean_dfs(
    dfs=tloads,
    this_var="zone-tloads",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=True,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    df_filter=df_filter,
)

In [ ]:
control_tloads_mean = base.run_passive_test_on_dfs(
    dfs=control_tloads, this_test="Mean", col_name="Zonal Load [%]<br>Control Days"
)
control_tloads_std = base.run_passive_test_on_dfs(
    dfs=control_tloads, this_test="Std", col_name="Zonal Load [%]<br>Control Days"
)

In [ ]:
dzs = copy.deepcopy(DOMINANT_ZONES)
for project in PROJECTS_2022:
    control_tloads_mean[project] = control_tloads_mean[project].dropna()
    control_tloads_std[project] = control_tloads_std[project].dropna()
control_tloads_mean, control_tloads_std, dzs = base.make_common_index(
    [control_tloads_mean, control_tloads_std, dzs]
)

In [ ]:
control_tloads_mean_tot = add_total_to_results_dict(control_tloads_mean)
control_tloads_std_tot = add_total_to_results_dict(control_tloads_std)
dzs_tot = add_total_to_results_dict(dzs)

In [ ]:
fig = viz.make_dot_plot(
    y_data=control_tloads_mean_tot,
    y_error_up_data=control_tloads_std_tot,
    y_error_down_data=control_tloads_std_tot,
    y_axis_title="Zonal Load [%]<br>Control Days",
    x_axis_title="Fraction of Zones [Unitless]",
    color_data=dzs_tot,
    color_legend={
        "name": {
            0: "Dominated",
            1: "Dominant",
            2: "Rogue",
        },
        "color": {
            0: DOMINANT_COLORS["Dominated"],
            1: DOMINANT_COLORS["Dominant"],
            2: DOMINANT_COLORS["Rogue"],
        },
    },
    normalize_x=True,
    num_cols=3,
    horizontal_spacing=0.1,
    vertical_spacing=0.1,
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
    y_range=[-120, 120],
    width=800,
    height=500,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.05,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC6.png")

## Persistence, cooling requests

In [ ]:
years = ["2022", "2023"]
months = [5, 6, 7, 8, 9]
periods = ["Frac Zones", " "]

for year in years:
    periods.append(year)
    for month in months:
        periods.append(f"{month}/{year}")
        if year == "2022" and month == 9:
            periods.append("  ")

groups = [
    "Dominated",
    "Dominant",
    "Rogue",
]

order_cols = [
    "Rogue",
    "Dominant",
    "Dominated",
]

persistence = {}

In [ ]:
for project in PROJECTS_2022:
    CRs_2022 = get_2022_control_data(
        project,
        "zone-simple_cooling_requests",
        no_weekends=True,
        only_business_hours=True,
        remove_FCUs=False,
        clean_underyling_data=True,  # cleans tload
    )
    CRs_2022 = CRs_2022.groupby(CRs_2022.index.month).sum()

    CRs_2023 = cleaning.clean_df(
        load_zones("2023", project, "zone-simple_cooling_requests", clean_data=True),
        this_var="zone-dummy",
        only_business_hours=True,
        no_weekends=True,
        remove_FCUs=False,
        start_date=pd.Timestamp("2023-05-01"),
        end_date=pd.Timestamp("2023-10-01"),
    )
    CRs_2023 = CRs_2023.groupby(CRs_2023.index.month).sum()

    this_dzs = DOMINANT_ZONES_TOT[project]
    CRs_2022, CRs_2023, this_dzs = base.trim_to_common_elements(
        [CRs_2022, CRs_2023, DOMINANT_ZONES_TOT[project].T],
        clean_cols=True,
        clean_idx=False,
    )
    CRs = {"2022": CRs_2022, "2023": CRs_2023}

    this_dzs = this_dzs.T
    all_zones = list(this_dzs.index)

    persistence_df = pd.DataFrame(0, index=periods, columns=groups)
    for group_i in range(len(groups)):
        group = groups[group_i]
        zones = list(
            (DOMINANT_ZONES_TOT[project][DOMINANT_ZONES_TOT[project] == group_i])
            .dropna()
            .index
        )
        persistence_df.loc["Frac Zones", group] = len(zones)
        # edge case correction
        zones = list(set(zones).intersection(set(all_zones)))
        for year in ["2022", "2023"]:
            persistence_df.loc[year, group] = CRs[year].loc[:, zones].sum().sum()
            for month in months:
                try:
                    persistence_df.loc[f"{month}/{year}", group] = (
                        CRs[year].loc[month, zones].sum()
                    )
                except:
                    persistence_df.loc[f"{month}/{year}", group] = np.nan
    persistence[project] = persistence_df[order_cols]
    print(project)

In [ ]:
persistence["TOTAL"] = pd.DataFrame(0, index=periods, columns=groups)
for project in PROJECTS_2022:
    persistence["TOTAL"] += persistence[project]

In [ ]:
for project in PROJECTS_2022_TOTAL:
    persistence[project] = persistence[project].div(
        persistence[project].sum(axis=1), axis=0
    )

In [ ]:
fig = viz.make_bar_plot(
    y_data=persistence,
    bar_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        },
    },
    y_axis_title="Fraction of Zones or<br>Cooling Requests<br>[Unitless]",
    y_range=[0, 1],
    bar_width=0.6,
    bar_mode="stack",
    num_cols=3,
    horizontal_spacing=0.125,
    vertical_spacing=0.1,
    width=800,
    height=500,
    title_size=MULTI_PLOT_TITLE_SIZE,
    legend_size=MULTI_PLOT_LEGEND_SIZE,
    text_size=MULTI_PLOT_TXT_SIZE,
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.075,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)
fig.update_xaxes(tickangle=45)

fig = add_line_to_subplots(
    fig,
    (1, 1),
    (0, 1.2),
    total_subplots=len(PROJECTS_2022_TOTAL),
    dash="solid",
    width=4,
)
fig = add_line_to_subplots(
    fig,
    (8, 8),
    (0, 1.2),
    total_subplots=len(PROJECTS_2022_TOTAL),
    dash="solid",
    width=4,
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC7.png")

## Change in CRs

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    only_business_hours=True,
    no_weekends=False,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)["temperature"]

In [ ]:
CRs = pull_from_dataset(
    "2022", PROJECTS_VAV, "zone-simple_cooling_requests", clean_data=True
)
CRs = cleaning.clean_dfs(
    dfs=CRs,
    this_var="zone-dummy",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
)

In [ ]:
categories = {
    0: "Dominated",
    1: "Dominant",
    2: "Rogue",
}

In [ ]:
CRs_grouped = {}
for project in PROJECTS_VAV:
    this_CRs_grouped = pd.DataFrame(
        index=CRs[project].index, columns=list(categories.values())
    )
    for i in categories:
        these_zones = list(
            DOMINANT_ZONES[project][DOMINANT_ZONES[project] == i].dropna().index
        )
        these_zones = list(
            set(these_zones).intersection(set(list(CRs[project].columns)))
        )
        this_CRs_grouped[categories[i]] = CRs[project][these_zones].sum(axis=1)
    CRs_grouped[project] = this_CRs_grouped

In [ ]:
(
    deltas_76_CRs,
    deltas_low_76_CRs,
    deltas_high_76_CRs,
    deltas_78_CRs,
    deltas_low_78_CRs,
    deltas_high_78_CRs,
) = run_group_regressions(CRs_grouped, T, "Absolute Change", summary_statistic="Mean")

In [ ]:
CRs_summary_df = pd.DataFrame(index=PROJECTS_VAV, columns=list(categories.values()))
for project in PROJECTS_VAV:
    for category in list(categories.values()):
        CRs_summary_df.loc[project, category] = deltas_78_CRs[project].loc[
            category, "78"
        ]

CRs_summary_df_low = pd.DataFrame(index=PROJECTS_VAV, columns=list(categories.values()))
for project in PROJECTS_VAV:
    for category in list(categories.values()):
        CRs_summary_df_low.loc[project, category] = deltas_low_78_CRs[project].loc[
            category, "78"
        ]

CRs_summary_df_high = pd.DataFrame(
    index=PROJECTS_VAV, columns=list(categories.values())
)
for project in PROJECTS_VAV:
    for category in list(categories.values()):
        CRs_summary_df_high.loc[project, category] = deltas_high_78_CRs[project].loc[
            category, "78"
        ]

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=CRs_summary_df,
    y_error_up_data=CRs_summary_df_high,
    y_error_down_data=CRs_summary_df_low,
    marker_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        },
    },
    y_axis_title="Absolute Change in<br>Cooling Requests Per Hour<br>[Requests/Hr]",
    point_start=(1 / 4),
    offset_delta=(1 / 4),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_VAV))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_VAV)],
    error_thickness=2.5,
    whisker_len=8,
    y_range=[-14, 4],
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_VAV) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC4.png")

## Change in airflow

In [ ]:
T = cleaning.clean_df(
    df=load_weather("2022")["temperature"].to_frame(),
    this_var="weather-oat",
    only_business_hours=True,
    no_weekends=False,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)["temperature"]

In [ ]:
airflow = pull_from_dataset("2022", PROJECTS_VAV, "zone-airflowsp")
airflow = cleaning.clean_dfs(
    dfs=airflow,
    this_var="zone-airflowsp",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
)

In [ ]:
categories = {
    0: "Dominated",
    1: "Dominant",
    2: "Rogue",
}

In [ ]:
airflow_grouped = {}
for project in PROJECTS_VAV:
    this_airflow_grouped = pd.DataFrame(
        index=airflow[project].index, columns=list(categories.values())
    )
    for i in categories:
        these_zones = list(
            DOMINANT_ZONES[project][DOMINANT_ZONES[project] == i].dropna().index
        )
        these_zones = list(
            set(these_zones).intersection(set(list(airflow[project].columns)))
        )
        this_airflow_grouped[categories[i]] = airflow[project][these_zones].sum(axis=1)
    airflow_grouped[project] = this_airflow_grouped

In [ ]:
(
    deltas_76_airflow_2022,
    deltas_low_76_airflow_2022,
    deltas_high_76_airflow_2022,
    deltas_78_airflow,
    deltas_low_78_airflow,
    deltas_high_78_airflow,
) = run_group_regressions(airflow_grouped, T, "Absolute Change")

In [ ]:
airflow_summary_df = pd.DataFrame(index=PROJECTS_VAV, columns=categories.values())
for project in PROJECTS_VAV:
    for category in categories.values():
        airflow_summary_df.loc[project, category] = deltas_78_airflow[project].loc[
            category, "78"
        ]

airflow_summary_df_low = pd.DataFrame(index=PROJECTS_VAV, columns=categories.values())
for project in PROJECTS_VAV:
    for category in categories.values():
        airflow_summary_df_low.loc[project, category] = deltas_low_78_airflow[
            project
        ].loc[category, "78"]

airflow_summary_df_high = pd.DataFrame(index=PROJECTS_VAV, columns=categories.values())
for project in PROJECTS_VAV:
    for category in categories.values():
        airflow_summary_df_high.loc[project, category] = deltas_high_78_airflow[
            project
        ].loc[category, "78"]

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=airflow_summary_df,
    y_error_up_data=airflow_summary_df_high,
    y_error_down_data=airflow_summary_df_low,
    marker_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        },
    },
    y_axis_title="Absolute Change in<br>Airflow [m3/Hr]",
    point_start=(1 / 4),
    offset_delta=(1 / 4),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_VAV))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_VAV)],
    error_thickness=2.5,
    whisker_len=8,
    y_range=[-25000, 6000],
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_VAV) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC3.png")

## Normalized airflow 

In [ ]:
categories = {
    0: "Dominated",
    1: "Dominant",
    2: "Rogue",
}

In [ ]:
airflow = pull_from_dataset("2022", PROJECTS_VAV, "zone-airflowsp")
airflow = cleaning.clean_dfs(
    dfs=airflow,
    this_var="zone-airflowsp",
    remove_FCUs=False,
    only_business_hours=True,
    no_weekends=NO_WEEKENDS,
    start_date=SUMMER_START_2022,
    end_date=SUMMER_END_2022,
    SI_units=True,
    hourly_filter=cleaning.get_experiment_hourly_filter(
        projects=PROJECTS_VAV,
        experiment_year="2022",
        filter_columns=["CSP = 74F"],
        no_weekends=True,
    ),
)

In [ ]:
airflow_grouped = {}
for project in PROJECTS_VAV:
    this_airflow_grouped = pd.DataFrame(
        index=airflow[project].index, columns=list(categories.values())
    )
    for i in categories:
        these_zones = list(
            DOMINANT_ZONES[project][DOMINANT_ZONES[project] == i].dropna().index
        )
        these_zones = list(
            set(these_zones).intersection(set(list(airflow[project].columns)))
        )
        this_airflow_grouped[categories[i]] = airflow[project][these_zones].sum(axis=1)
    airflow_grouped[project] = this_airflow_grouped

for project in PROJECTS_VAV:
    airflow_grouped[project] = airflow_grouped[project].div(
        airflow_grouped[project].sum(axis=1), axis=0
    )

In [ ]:
airflow_summary_df = pd.DataFrame(index=PROJECTS_VAV, columns=categories.values())
for project in PROJECTS_VAV:
    for category in categories.values():
        airflow_summary_df.loc[project, category] = airflow_grouped[project][
            category
        ].mean()

airflow_summary_df_low = pd.DataFrame(index=PROJECTS_VAV, columns=categories.values())
for project in PROJECTS_VAV:
    for category in categories.values():
        airflow_summary_df_low.loc[project, category] = airflow_grouped[project][
            category
        ].std()

airflow_summary_df_high = pd.DataFrame(index=PROJECTS_VAV, columns=categories.values())
for project in PROJECTS_VAV:
    for category in categories.values():
        airflow_summary_df_high.loc[project, category] = airflow_grouped[project][
            category
        ].std()

In [ ]:
fig = viz.plot_experiment_summary(
    y_data=airflow_summary_df,
    y_error_up_data=airflow_summary_df_high,
    y_error_down_data=airflow_summary_df_low,
    marker_legend={
        "color": {
            "Dominated": DOMINANT_COLORS["Dominated"],
            "Dominant": DOMINANT_COLORS["Dominant"],
            "Rogue": DOMINANT_COLORS["Rogue"],
        },
    },
    y_axis_title="Fraction of Building-Wide Airflow<br>Control Days [Unitless]",
    point_start=(1 / 4),
    offset_delta=(1 / 4),
    tick_vals=[i + 0.5 for i in range(len(PROJECTS_VAV))],
    width=1200,
    height=600,
    x_range=[-0.25, len(PROJECTS_VAV)],
    error_thickness=2.5,
    whisker_len=8,
    y_range=[-0.1, 1],
    legend_size=SINGLE_PLOT_LEGEND_SIZE,
    text_size=SINGLE_PLOT_TXT_SIZE,
)

fig = add_vertical_boxes(
    fig, list(range(len(PROJECTS_VAV) + 1)), background_color="lightgray"
)
fig = fig.update_layout(
    legend=dict(
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        orientation="h",
    )
)

In [ ]:
# fig

In [ ]:
fig.write_image(f"{IMAGE_PATH}/FigureC8.png")